In [2]:
import os
import numpy as np
from PIL import Image
import io
import re
import zipfile

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

# import fiftyone as fo
# import fiftyone.zoo as foz

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "bigbucket"  # @param {type:"string"}
from datetime import datetime
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse
!mkdir -p mush
bucket_name='medium_mush'
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name} mush

Project ID:  
deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  93962      0 --:--:-- --:--:-- --:--:-- 93962
OK
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [770 kB]
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRel

In [3]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect() # TPU detection
strategy = tf.distribute.TPUStrategy(tpu)

INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Deallocate tpu buffers before initializing tpu system.


INFO:tensorflow:Initializing the TPU system: grpc://10.69.168.122:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.69.168.122:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [22]:
import pandas as pd

# VALIDATION_BATCH_SIZE = 256
# BATCH_SIZE=2
# EPOCHS=2
# AUTOTUNE = tf.data.AUTOTUNE
# IMAGE_SIZE = [331, 331]

# if strategy.num_replicas_in_sync == 8: # TPU or 8xGPU
#     BATCH_SIZE = 16 * strategy.num_replicas_in_sync
#     VALIDATION_BATCH_SIZE = 16 * strategy.num_replicas_in_sync
#     start_lr = 0.00001
#     min_lr = 0.00001
#     max_lr = 0.00005 * strategy.num_replicas_in_sync
#     rampup_epochs = 5
#     sustain_epochs = 0
#     exp_decay = .8

# MIXED_PRECISION = True
# if MIXED_PRECISION:
#     if tpu: 
#         policy = tf.keras.mixed_precision.Policy('mixed_bfloat16')
#     else: #
#         policy = tf.keras.mixed_precision.Policy('mixed_float16')
#         tf.config.optimizer.set_jit(True) # XLA compilation
#     tf.keras.mixed_precision.set_global_policy(policy)
#     print('Mixed precision enabled')
#### EXPERIMENT PARAMETERS

# TENSORBOARD_LOG_DIR   : "/data-ssd/alex/experiments/slim_export/1608157217/log_dir"
# CHECKPOINT_DIR        : "/data-ssd/alex/experiments/slim_export/1608157217/checkpoints/ckpt"
# FINAL_SAVE_DIR        : "/data-ssd/alex/experiments/slim_export/1608157217/final_model/final"
# BACKUP_DIR            : "/data-ssd/alex/experiments/slim_export/1608157217/backup"


#### DATASET PARAMETERS

TRAINING_DATA         = "/content/t.csv"
VAL_DATA              = "/content/val.csv"
# TEST_DATA             = "/data-ssd/alex/datasets/slim_export_20201213/test_cleaned.json"
NUM_CLASSES           = 1394


#### TRAINING PARAMETERS

# training policy - only use mixed precision=true if you
# have a recent NVIDIA GPU that supports CUDA 7.0 or later
TRAIN_MIXED_PRECISION = True

# size of batch, per gpu
BATCH_SIZE            = 256

# number of training epochs
NUM_EPOCHS            = 10

# initial learning rate for the model
INITIAL_LEARNING_RATE = 0.05
LR_DECAY_FACTOR       = 0.94
EPOCHS_PER_LR_DECAY   = 4
PRETRAINED_MODEL = False

#### MODEL PARAMETERS

# neural network architecture
MODEL_NAME            = "xception"

# size of input
IMAGE_SIZE            = (299,299)

# dropout percentage for layer between pool & logits
DROPOUT_PCT           = 0.5

# optiimzer
OPTIMIZER_NAME  = "rmsprop"
RMSPROP_RHO     = 0.9
RMSPROP_MOMENTUM= 0.9
RMSPROP_EPSILON = 1.0

MULTIGPU = False
DO_LABEL_SMOOTH = False

In [ ]:
def lrfn(epoch):
    def lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay):
        if epoch < rampup_epochs:
            lr = (max_lr - start_lr)/rampup_epochs * epoch + start_lr
        elif epoch < rampup_epochs + sustain_epochs:
            lr = max_lr
        else:
            lr = (max_lr - min_lr) * exp_decay**(epoch-rampup_epochs-sustain_epochs) + min_lr
        return lr
    return lr(epoch, start_lr, min_lr, max_lr, rampup_epochs, sustain_epochs, exp_decay)

lr_callback = tf.keras.callbacks.LearningRateScheduler(lambda epoch: lrfn(epoch), verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, [lrfn(x) for x in rng])
print(y[0], y[-1])

def display_training_curves(training, validation, title, subplot):
  if subplot%10==1: # set up the subplots on the first call
    plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
    #plt.tight_layout() # bug in tight layout in this version of matplotlib
  ax = plt.subplot(subplot)
  ax.set_facecolor('#F8F8F8')
  ax.plot(training)
  ax.plot(validation)
  ax.set_title('model '+ title)
  ax.set_ylabel(title)
  #ax.set_ylim(0.28,1.05)
  ax.set_xlabel('epoch')
  ax.legend(['train', 'valid.'])

  def force_image_sizes(dataset, image_size):
    # explicit size will be needed for TPU
    reshape_images = lambda image, label: (tf.reshape(image, [*image_size, 3]), label)
    dataset = dataset.map(reshape_images, num_parallel_calls=AUTOTUNE)
    return dataset

NameError: ignored

In [ ]:
def create_model():
    #pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    pretrained_model = tf.keras.applications.Xception(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    pretrained_model.trainable = True

    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(5, activation='softmax', dtype=tf.float32) # the float32 is needed on softmax layer when using mixed precision
    ])

    model.compile(
        optimizer='adam',
        loss = 'categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

with strategy.scope(): # creating the model in the TPUStrategy scope places the model on the TPU
    model = create_model()
model.summary()

83697664/83683744 [==============================] - 1s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 11, 11, 2048)      20861480  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 5)                 10245     
                                                                 
Total params: 20,871,725
Trainable params: 20,817,197
Non-trainable params: 54,528
_________________________________________________________________


In [ ]:
# def get_training_dataset():
#     # dataset = load_dataset(TRAIN_FILENAMES)
#     # dataset = dataset.map(data_augment, num_parallel_calls=AUTOTUNE)
#     dataset = dataset.repeat()
#     dataset = dataset.shuffle(2048)
#     dataset = dataset.batch(BATCH_SIZE)
#     dataset = dataset.prefetch(AUTOTUNE) # prefetch next batch while training (autotune prefetch buffer size)
#     return dataset

# def get_validation_dataset():
#     # dataset = load_dataset(VALID_FILENAMES)
#     dataset = dataset.batch(VALIDATION_BATCH_SIZE)
#     dataset = dataset.prefetch(AUTOTUNE)

In [ ]:
train['full_path'] = 'content/mush/fungi/'+train['file_name']
val['full_path'] = 'content/mush/fungi/'+val['file_name']


# def parse_image(filename):
#     file = tf.io.read_file(filename) # this will work only with filename as tensor
#     image = tf.image.decode_image(file)
# #     return image

# def load(file_path, label):
#     img = tf.io.read_file(file_path)
#     img = tf.image.decode_png(img, channels=3)
#     img = tf.image.convert_image_dtype(img, tf.float32)
#     img = tf.image.resize(img, size=IMAGE_SIZE) # optional
#     label = tf.cast(tf.equal(label, 'class2'), tf.int32)
#     return img, label

# ds = tf.data.Dataset.from_tensor_slices((X, y)).map(lambda x, y: load(X, y))

# next(iter(ds))



# dataset = tf.data.Dataset.from_tensor_slices(X)
# dataset.
# dataset = dataset.map(lambda x, y:dataset = dataset.map(parse_image).batch(1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
train = pd.read_csv('/content/mush/fungi/training_annot.csv')
val = pd.read_csv('/content/mush/fungi/val_annot.csv')

#dataset = tf.data.Dataset.from_tensor_slices((train['file_name'], train['category_id']))

In [16]:
from functools import partial

AUTOTUNE = tf.data.experimental.AUTOTUNE

# augments
# currently not using rotate
def _rotate(x: tf.Tensor, y: tf.Tensor) -> (tf.Tensor, tf.Tensor):
    rotate_amt = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    x = tf.image.rot90(x, rotate_amt)
    return x, y


def _flip(x: tf.Tensor, y: tf.Tensor) -> (tf.Tensor, tf.Tensor):
    x = tf.image.random_flip_left_right(x)
    # right left only
    # x = tf.image.random_flip_up_down(x)
    return x, y


def _color(x: tf.Tensor, y: tf.Tensor) -> (tf.Tensor, tf.Tensor):
    x = tf.image.random_hue(x, 0.08)
    x = tf.image.random_saturation(x, 0.6, 1.6)
    x = tf.image.random_brightness(x, 0.05)
    x = tf.image.random_contrast(x, 0.7, 1.3)
    return x, y


def _random_crop(x: tf.Tensor, y: tf.Tensor) -> (tf.Tensor, tf.Tensor):
    bbox = tf.constant([0.0, 0.0, 1.0, 1.0], dtype=tf.float32, shape=[1, 1, 4])

    begin, size, bbox_for_draw = tf.image.sample_distorted_bounding_box(
        tf.shape(x),
        bounding_boxes=bbox,
        area_range=(0.08, 1.0),
        aspect_ratio_range=(0.75, 1.33),
        max_attempts=100,
        min_object_covered=0.1,
    )
    x = tf.slice(x, begin, size)

    return x, y


def _decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_jpeg(img, channels=3)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # no resizing, we may augment which will crop.
    # we resize _after_ the augments pass
    return img


def process_row(file_path, label, num_classes):
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = _decode_img(img)
    # 1 hot encode the label for dense
    label = tf.one_hot(label, num_classes)
    return img, label


def _load_dataframe(dataset_json_path):
    df = pd.read_csv(dataset_json_path)

    # sort the dataset
    df = df.sample(frac=1, random_state=42)
    return df


def _prepare_dataset(
    ds,
    image_size=(299, 299),
    batch_size=32,
    repeat_forever=True,
    shuffle_buffer_size=10000,
    augment=False,
):
    # shuffle
    ds = ds.shuffle(buffer_size=shuffle_buffer_size)

    # do transforms for augment or not
    if augment:
        # crop 100% of the time
        ds = ds.map(lambda x, y: _random_crop(x, y), num_parallel_calls=AUTOTUNE)
        # flip 50% of the time
        # the function already flips 50% of the time, so we call it 100% of the time
        ds = ds.map(lambda x, y: _flip(x, y), num_parallel_calls=AUTOTUNE)
        # do color 30% of the time
        ds = ds.map(
            lambda x, y: tf.cond(
                tf.random.uniform([], 0, 1) > 0.7, lambda: _color(x, y), lambda: (x, y)
            ),
            num_parallel_calls=AUTOTUNE,
        )
        # resize to image size expected by network
        ds = ds.map(lambda x, y: (tf.image.resize(x, image_size), y))
        # make sure the color transforms haven't move any of the pixels outside of [0,1]
        ds = ds.map(
            lambda x, y: (tf.clip_by_value(x, 0, 1), y), num_parallel_calls=AUTOTUNE
        )
    else:
        # central crop
        # ds = ds.map(lambda x,y: (tf.image.central_crop(x, 0.875), y))
        # resize to image size expected by network
        ds = ds.map(lambda x, y: (tf.image.resize(x, image_size), y))

    # Repeat forever
    if repeat_forever:
        ds = ds.repeat()

    ds = ds.batch(batch_size)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds


def make_dataset(
    path,
    label_column_name,
    image_size=(299, 299),
    batch_size=BATCH_SIZE,
    repeat_forever=True,
    augment=False,
):
    df = _load_dataframe(path)
    num_examples = len(df)
    num_classes = len(df[label_column_name].unique())

    ds = tf.data.Dataset.from_tensor_slices((df["full_path"], df[label_column_name]))

    process_partial = partial(process_row, num_classes=num_classes)
    ds = ds.map(process_partial, num_parallel_calls=AUTOTUNE)

    ds = _prepare_dataset(
        ds,
        image_size=image_size,
        batch_size=batch_size,
        repeat_forever=repeat_forever,
        augment=augment,
    )

    return (ds, num_examples)

In [ ]:
training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()

In [ ]:
start_time = time.time()
history = model.fit(training_dataset, validation_data=validation_dataset,
                    steps_per_epoch=TRAIN_STEPS, epochs=EPOCHS, callbacks=[lr_callback])

final_accuracy = history.history["val_accuracy"][-5:]
print("FINAL ACCURACY MEAN-5: ", np.mean(final_accuracy))
print("TRAINING TIME: ", time.time() - start_time, " sec")

In [ ]:
import os
import time
import pandas as pd
import numpy as np
import argparse
import yaml

import json

import tensorflow as tf
from tensorflow import keras
AUTOTUNE = tf.data.experimental.AUTOTUNE

# def make_training_callbacks(config):

def lr_scheduler_fn(epoch):
    return INITIAL_LEARNING_RATE * \
        tf.math.pow(LR_DECAY_FACTOR, epoch//EPOCHS_PER_LR_DECAY)
callbacks = [
            tf.keras.callbacks.LearningRateScheduler(
            lr_scheduler_fn,
            verbose=1
),
     ]
# return callbacks
# keras.callbacks.TensorBoard(
#     log_dir=config["TENSORBOARD_LOG_DIR"],
#     histogram_freq=0,
#     write_graph=False,
#     write_images=False,
#     update_freq=20,
#     profile_batch=0,
#     embeddings_freq=0,
#     embeddings_metadata={}
#),
# policy = tf.keras.mixed_precision.Policy('mixed_bfloat16')
# tf.keras.mixed_precision.set_global_policy(policy)
# tf.keras.callbacks.ModelCheckpoint(
#     filepath=config["CHECKPOINT_DIR"],
#     save_weights_only=True,
#     save_best_only=True,
#     monitor="val_accuracy",
#     verbose=1
# ),
# tf.keras.callbacks.experimental.BackupAndRestore(
#     backup_dir=config["BACKUP_DIR"],
# ),

    # def main():
# get command line args
# parser = argparse.ArgumentParser(description="Train an iNat model.")
# parser.add_argument(
#     '--config_file',
#     required=True,
#     help="YAML config file for training."
# )
# args = parser.parse_args()
# read in config file
# if not os.path.exists(args.config_file):
#     print("No config file.")
#     return
# with open(args.config_file, "r") as f:
#     config = yaml.safe_load(f)

# if TRAIN_MIXED_PRECISION:
#     from tensorflow.keras.mixed_precision import experimental as mixed_precision
#     policy = mixed_precision.Policy('mixed_bfloat16')
#     mixed_precision.set_policy(policy)

# if MULTIGPU:
#     strategy = tf.distribute.MirroredStrategy()
# else:
#     strategy = tf.distribute.get_strategy()

# load train & val datasets
if not os.path.exists(TRAINING_DATA):
    print("Training data file doesn't exist.")
    
(train_ds, num_train_examples) = make_dataset(
    TRAINING_DATA,
    label_column_name='category_id',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    repeat_forever=True,
    augment=False
)
if train_ds is None:
    print("No training dataset.")

    
if num_train_examples == 0:
    print("No training examples.")
    

if not os.path.exists(VAL_DATA):
    print("Validation data file doesn't exist.")
    
(val_ds, num_val_examples) = make_dataset(
    VAL_DATA,
    label_column_name='category_id',
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    repeat_forever=True,
    augment=False
)
if val_ds is None:
    print("No val dataset.")
    
if num_val_examples == 0:
    print("No val examples.")
    
    # load pretrained model
    if False and PRETRAINED_MODEL != "imagenet" and os.path.exists(PRETRAINED_MODEL):
         model.load_weights(PRETRAINED_MODEL, by_name=True)
    if model is None:
        print("No model to train.")
        
if DO_LABEL_SMOOTH:
    if LABEL_SMOOTH_MODE == "flat":
         # with flat label smoothing we can do it all
         # in the loss function
        loss=tf.keras.losses.CategoricalCrossentropy(
        label_smoothing=LABEL_SMOOTH_PCT
        )
    else:
        # with parent/heirarchical label smoothing
        # we can't do it in the loss function, we have
        # to adjust the labels in the dataset
        print("Unsupported label smoothing mode.")
        
else:
    loss=tf.keras.losses.CategoricalCrossentropy()
     # compile the network for training
with strategy.scope():
    # create optimizer for neural network
    optimizer = keras.optimizers.RMSprop(
        lr=INITIAL_LEARNING_RATE,
        rho=RMSPROP_RHO,
        momentum=RMSPROP_MOMENTUM,
        epsilon=RMSPROP_EPSILON
    )    # create neural network
    model = make_neural_network(
        base_arch_name = "xception",
        weights = 'imagenet',
        image_size = IMAGE_SIZE,
        dropout_pct = DROPOUT_PCT,
        n_classes = NUM_CLASSES,
        input_dtype = 'float32',
        train_full_network = True)
    model.compile(
        loss=loss,
        optimizer=optimizer,
        metrics=[
            "accuracy", 
            tf.keras.metrics.TopKCategoricalAccuracy(k=3, name="top3 accuracy"),
            tf.keras.metrics.TopKCategoricalAccuracy(k=10, name="top10 accuracy")
    ]
)
 # setup callbacks
# training_callbacks = make_training_callbacks(config)
STEPS_PER_EPOCH = np.ceil(num_train_examples/BATCH_SIZE)
VAL_STEPS = np.ceil(num_val_examples/BATCH_SIZE)
start = time.time()
hisory = model.fit(
    train_ds,
    validation_data=val_ds,
    validation_steps=VAL_STEPS,
    epochs=NUM_EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=callbacks
)
end = time.time()
print("time elapsed during fit: {:.1f}".format(end-start))
print(history.history)
# model.save(FINAL_SAVE_DIR)
# if __name__ == "__main__":
#     main()


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


83697664/83683744 [==============================] - 1s 0us/step

Epoch 1: LearningRateScheduler setting learning rate to 0.05000000074505806.
Epoch 1/10
 17/335 [>.............................] - ETA: 1:14:45 - loss: 7.2201 - accuracy: 0.0025 - top3 accuracy: 0.0080 - top10 accuracy: 0.0218

In [ ]:
|import pandas as pd

In [7]:
train = train[['file_name', 'category_id']]
train['full_path'] = 'gs://medium_mush/fungi/' + train['file_name']
val = val[['file_name', 'category_id']]
val['full_path'] = 'gs://medium_mush/fungi/' + val['file_name']
val.to_csv('val.csv', index=False)
train.to_csv('t.csv', index=False)

In [ ]:
val = pd.read_csv('/content/mush/fungi/val_annot.csv')

In [9]:
def make_neural_network(base_arch_name, weights, image_size, dropout_pct, n_classes, input_dtype, train_full_network):
    image_size_with_channels = image_size 
    base_arch = keras.applications.Xception if base_arch_name == "xception" else None
    if not base_arch:
        print("Unsupported base architecture.")
        return None

    input_layer = keras.layers.Input(shape=(299, 299, 3), dtype=input_dtype)
    base_model = base_arch(input_tensor=input_layer, weights=weights, include_top=False)
    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    dropout = keras.layers.Dropout(dropout_pct)(avg)
    x = keras.layers.Dense(1394, name="dense_logits")(dropout)
    output = keras.layers.Activation("softmax", dtype="float32", name="predictions")(x)
    model = keras.Model(inputs=base_model.input, outputs=output)

    if train_full_network:
        for layer in base_model.layers:
            layer.trainable = True
    return model

In [ ]:
len(train['category_id'].unique())

1394

In [ ]:
        start = time.time()
        history = model.fit(
            train_ds,
            validation_data=val_ds,
            validation_steps=VAL_STEPS,
            epochs=NUM_EPOCHS,
            steps_per_epoch=STEPS_PER_EPOCH,
            callbacks=training_callbacks
        )

        end = time.time()

NameError: ignored

In [ ]:
val_ds

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 299, 299, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1394), dtype=tf.float32, name=None))>

In [8]:
val

,file_name,category_id,full_path
0,images/16390_Leucoagaricus_leucothites/JM2009P...,650,gs://medium_mush/fungi/images/16390_Leucoagari...
1,images/16390_Leucoagaricus_leucothites/JM2012P...,650,gs://medium_mush/fungi/images/16390_Leucoagari...
2,images/16390_Leucoagaricus_leucothites/MC2009P...,650,gs://medium_mush/fungi/images/16390_Leucoagari...
3,images/40985_Byssomerulius_corium/MC2009PIC320...,1158,gs://medium_mush/fungi/images/40985_Byssomerul...
4,images/40985_Byssomerulius_corium/MC2017-91902...,1158,gs://medium_mush/fungi/images/40985_Byssomerul...
...,...,...,...
4177,images/16365_Leptosporomyces_fuscostratus/ML20...,648,gs://medium_mush/fungi/images/16365_Leptosporo...
4178,images/16365_Leptosporomyces_fuscostratus/ML20...,648,gs://medium_mush/fungi/images/16365_Leptosporo...
4179,images/16382_Leucoagaricus_leucothites var. ca...,649,gs://medium_mush/fungi/images/16382_Leucoagari...
4180,images/16382_Leucoagaricus_leucothites var. ca...,649,gs://medium_mush/fungi/images/16382_Leucoagari...
